In [1]:
import numpy as np  
import pandas as pd  
from jproperties import Properties

import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import make_pipeline
import multiprocessing
import seaborn as sns
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
configs = Properties()
with open('config.properties', 'rb') as config_file:
    configs.load(config_file)
csvfilepath = configs["filePath"].data

        # Read the csv File
df = pd.read_csv(csvfilepath, na_values='')
#objectProductReco = ProdReco(csvfilepath)
print(df.shape)

#correlation matrix
#corrmat = objectProductReco.corr()
#f, ax = plt.subplots(figsize=(12, 9))
#sns.heatmap(corrmat, vmax=.8, square=True);



(384, 25)


In [3]:
df.dtypes

PERSON_ID                 int64
PERSON_TYPE              object
BIRTH_DT                 object
LANGUAGE_CD              object
DIVISION                 object
CUSTOMER_SEG_FLG         object
CUSTOMER_TIER_FLG        object
PERSON_NAME              object
PARTY_TYPE               object
PRICING_ATTACHED_TO      object
PRICEITEM_CD             object
PRICELIST_ID            float64
PRICE_STATUS             object
PRICING_START_DT         object
PRICING_END_DT          float64
PRICE_ASGN_ID           float64
ACCOUNT_ID                int64
CUST_CL_CD               object
CIS_DIVISION             object
ACCOUNT_CATEGORY         object
SETUP_DT                 object
CURRENCY_CD              object
BILL_CYC_CD              object
PRODUCT_CD               object
PRODUCT_EFFECTIVE_DT     object
dtype: object

In [4]:
str_feature_types = df.dtypes.astype(str).value_counts()
str_feature_types

object     20
float64     3
int64       2
dtype: int64

In [5]:
#Find the null value columns
null_columns=df.columns[df.isnull().any()]

df[null_columns].isnull().sum()

BIRTH_DT                359
PARTY_TYPE               31
PRICELIST_ID             85
PRICING_END_DT          384
PRICE_ASGN_ID            31
PRODUCT_CD              353
PRODUCT_EFFECTIVE_DT    353
dtype: int64

In [6]:
thresh = len(df) * .2
df.dropna(thresh = thresh, axis = 1, inplace = True)

In [7]:
df.dtypes

PERSON_ID                int64
PERSON_TYPE             object
LANGUAGE_CD             object
DIVISION                object
CUSTOMER_SEG_FLG        object
CUSTOMER_TIER_FLG       object
PERSON_NAME             object
PARTY_TYPE              object
PRICING_ATTACHED_TO     object
PRICEITEM_CD            object
PRICELIST_ID           float64
PRICE_STATUS            object
PRICING_START_DT        object
PRICE_ASGN_ID          float64
ACCOUNT_ID               int64
CUST_CL_CD              object
CIS_DIVISION            object
ACCOUNT_CATEGORY        object
SETUP_DT                object
CURRENCY_CD             object
BILL_CYC_CD             object
dtype: object

In [8]:
#Removing columns which are not required
df_processed1=df.drop(['PRICING_START_DT','SETUP_DT','BILL_CYC_CD','PERSON_NAME'],axis=1)

In [9]:
#Remove id /code columns- data maintaining columns
df_processed1=df_processed1.drop(['PRICELIST_ID','ACCOUNT_ID','PRICE_ASGN_ID'],axis=1)

In [10]:
df_processed1.dtypes

PERSON_ID               int64
PERSON_TYPE            object
LANGUAGE_CD            object
DIVISION               object
CUSTOMER_SEG_FLG       object
CUSTOMER_TIER_FLG      object
PARTY_TYPE             object
PRICING_ATTACHED_TO    object
PRICEITEM_CD           object
PRICE_STATUS           object
CUST_CL_CD             object
CIS_DIVISION           object
ACCOUNT_CATEGORY       object
CURRENCY_CD            object
dtype: object

In [11]:
list(df_processed1)

['PERSON_ID',
 'PERSON_TYPE',
 'LANGUAGE_CD',
 'DIVISION',
 'CUSTOMER_SEG_FLG',
 'CUSTOMER_TIER_FLG',
 'PARTY_TYPE',
 'PRICING_ATTACHED_TO',
 'PRICEITEM_CD',
 'PRICE_STATUS',
 'CUST_CL_CD',
 'CIS_DIVISION',
 'ACCOUNT_CATEGORY',
 'CURRENCY_CD']

In [12]:
column_list = ['PERSON_ID',
 'PERSON_TYPE',
 'LANGUAGE_CD',
 'DIVISION',
 'CUSTOMER_SEG_FLG',
 'CUSTOMER_TIER_FLG',
 'PERSON_NAME',
 'PARTY_TYPE',
 'PRICING_ATTACHED_TO',
 'PRICEITEM_CD',
 'PRICE_STATUS',
 'CUST_CL_CD',
 'CIS_DIVISION',
 'ACCOUNT_CATEGORY',
 'CURRENCY_CD']


dataset = df_processed1.reindex(columns=column_list)

In [13]:
dataset_person_item =df_processed1.groupby(by=['PERSON_ID', 'PRICEITEM_CD'],as_index=False).first()
dataset_person_item

,PERSON_ID,PRICEITEM_CD,PERSON_TYPE,LANGUAGE_CD,DIVISION,CUSTOMER_SEG_FLG,CUSTOMER_TIER_FLG,PARTY_TYPE,PRICING_ATTACHED_TO,PRICE_STATUS,CUST_CL_CD,CIS_DIVISION,ACCOUNT_CATEGORY,CURRENCY_CD
0,72231957,AB_AOF,P,ENG,CAL,CORP,,PERS,Price List,ACTV,BKCORP,CAL,USAG,USD
1,72231957,ACCMAINT,P,ENG,CAL,CORP,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD
2,72231957,DISCT_KS,P,ENG,CAL,CORP,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD
3,72231957,DM_P1,P,ENG,CAL,CORP,,PERS,Price List,ACTV,BKCORP,CAL,USAG,USD
4,156312464,CHQTFR,P,ENG,NY,CORP,T1,PERS,Price List,ACTV,BKCORP,NY,USAG,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,8640624497,DOM_MIN,P,ENG,CAL,,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD
171,8640624497,DOM_STD,P,ENG,CAL,,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD
172,8640624497,DOM_URG,P,ENG,CAL,,,PERS,Account/Person,ACTV,BKCORP,CAL,USAG,USD
173,9541308934,PP_PI,P,ENG,CAL,,,ACCT,Account/Person,ACTV,BKCORP,CAL,USAG,USD


In [14]:
item_count =df_processed1.groupby(['PERSON_ID', 'PRICEITEM_CD']).size().reset_index().rename(columns={0:'ItemCount'})
item_count

,PERSON_ID,PRICEITEM_CD,ItemCount
0,72231957,AB_AOF,1
1,72231957,ACCMAINT,1
2,72231957,DISCT_KS,1
3,72231957,DM_P1,2
4,156312464,CHQTFR,1
...,...,...,...
170,8640624497,DOM_MIN,3
171,8640624497,DOM_STD,3
172,8640624497,DOM_URG,3
173,9541308934,PP_PI,4


In [15]:
tobe_processed =dataset_person_item.iloc[:,2:]

In [16]:

MAX_OH_CARDINALITY=10
def select_oh_features(dataset):
    
    hc_features =dataset\
        .select_dtypes(['object', 'category'])\
        .apply(lambda col: col.nunique())\
        .loc[lambda x: x <= MAX_OH_CARDINALITY]\
        .index\
        .tolist()
        
    return hc_features

oh_features = select_oh_features(dataset)

print(f'N oh_features: {len(oh_features)} \n')
print(', '.join(oh_features))
 

N oh_features: 12 

PERSON_TYPE, LANGUAGE_CD, DIVISION, CUSTOMER_SEG_FLG, CUSTOMER_TIER_FLG, PARTY_TYPE, PRICING_ATTACHED_TO, PRICE_STATUS, CUST_CL_CD, CIS_DIVISION, ACCOUNT_CATEGORY, CURRENCY_CD


In [17]:
oh_pipeline = make_pipeline(SimpleImputer(strategy='constant'), OneHotEncoder(handle_unknown='ignore'))

In [18]:
column_transformer = ColumnTransformer(transformers=\
                                       [('oh_pipeline', oh_pipeline, select_oh_features)],
                                       n_jobs = multiprocessing.cpu_count(),
                                       remainder='drop')

In [19]:
processed_data = column_transformer.fit_transform(tobe_processed)
processed_dataframe = pd.DataFrame(processed_data)

In [20]:
person_col = item_count['PERSON_ID']
item_col = item_count['PRICEITEM_CD']
itemcount_col = item_count['ItemCount']
processed_dataframe.insert(0, 'PERSON_ID', person_col, True)
processed_dataframe.insert(1, 'PRICEITEM_CD', item_col, True)
processed_dataframe.insert(2, 'ITEM_COUNT', itemcount_col, True)

In [21]:

processed_dataframe

,PERSON_ID,PRICEITEM_CD,ITEM_COUNT,0,1,2,3,4,5,6,...,26,27,28,29,30,31,32,33,34,35
0,72231957,AB_AOF,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,72231957,ACCMAINT,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,72231957,DISCT_KS,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,72231957,DM_P1,2,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,156312464,CHQTFR,1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,8640624497,DOM_MIN,3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
171,8640624497,DOM_STD,3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
172,8640624497,DOM_URG,3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
173,9541308934,PP_PI,4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
#Classifier 
from sklearn.linear_model import LogisticRegression
preds = LogisticRegression().fit(digits.data, digits.target).predict_proba(digits.data)
print([i[1] for i in preds]) 

In [70]:
value_generation =processed_dataframe.iloc[:,2:]

In [71]:
result = (value_generation*value_generation).sum(axis=1)
result

0      13.0
1      13.0
2      13.0
3      16.0
4      13.0
       ... 
170    21.0
171    21.0
172    21.0
173    28.0
174    28.0
Length: 175, dtype: float64

In [72]:
person_item_data = pd.DataFrame(list(zip(person_col,item_col, result)), 
               columns =['PERSON_ID','PRICEITEM_CD', 'Score']) 

In [73]:
person_item_data

,PERSON_ID,PRICEITEM_CD,Score
0,72231957,AB_AOF,13.0
1,72231957,ACCMAINT,13.0
2,72231957,DISCT_KS,13.0
3,72231957,DM_P1,16.0
4,156312464,CHQTFR,13.0
...,...,...,...
170,8640624497,DOM_MIN,21.0
171,8640624497,DOM_STD,21.0
172,8640624497,DOM_URG,21.0
173,9541308934,PP_PI,28.0


In [82]:
def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    :param :data_items
    """

    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    similarity_data = pd.DataFrame(data=similarities, index=data_items.columns, columns=data_items.columns)
    return similarity_data

In [83]:
person_item_data = pd.DataFrame(list(zip(person_col, item_col, result)),
                                columns=['PERSON_ID', 'PRICEITEM_CD', 'Score'])
#print(person_item_data)
TableCustItem = person_item_data.pivot(index='PERSON_ID', columns='PRICEITEM_CD', values='Score')
TableCustItem.shape
TableCustItem = TableCustItem.replace(np.nan, 0)
TableCustItem = TableCustItem.merge(person_col, on='PERSON_ID', how='left')
print(TableCustItem.shape)
dfcross_tabed1 = TableCustItem.drop('PERSON_ID', 1)
# Normalizing the values
magnitude = np.sqrt(np.square(dfcross_tabed1).sum(axis=1))
dfcross_tabed2 = dfcross_tabed1.divide(magnitude, axis='index')
# Calling the similarity Function for Item-Item Matrix
data_matrix = calculate_similarity(dfcross_tabed2)
# Changing the datatype of Person_Id to str
TableCustItem['PERSON_ID'] = TableCustItem.PERSON_ID.astype(str)
data_items = TableCustItem.drop('PERSON_ID', 1)
data_items2 = data_items.divide(magnitude, axis='index')
scoredata = data_matrix.dot(data_items2.T)
scoredata = scoredata.T
scoredata = scoredata.div(data_matrix.sum(axis=1))
bought_df = data_items2.where(data_items2 == 0, -999999)
recommend_df = bought_df.where(bought_df != 0, scoredata)
new_col = TableCustItem['PERSON_ID']
recommend_df.insert(0, 'PERSON_ID', new_col, True)
# recommd_df -have to save in a table/datastore
#recommend_df.to_json("recommendation1.json", orient="columns")

(175, 85)


In [84]:
    def recommended_execution(customer_no, no_products,recommend_df):
        """Give the product recommendation for a customer based on the input .
                     """
        recommd_df = pd.read_json(r'recommendation1.json')
        recommd_df['PERSON_ID'] = recommd_df.PERSON_ID.astype(str)
        data_items = recommd_df.drop('PERSON_ID', 1)
        user_index = recommd_df[recommd_df.PERSON_ID == customer_no].index.tolist()[0]
        rating = data_items.loc[user_index]
        recommend_products_series = rating.nlargest(no_products)
        print(recommend_products_series)
        return recommend_products_series

In [86]:
recommended_execution('72231957', 10,recommend_df)

SIV123                            0.140681
DM_P2                             0.117106
AB_CCARDS                         0.076253
AB_CWN                            0.052254
AB_AMF                            0.048927
ACCOPENING                        0.040577
PAY_DOM_BTB                       0.040577
PAY_DOM_URG                       0.040577
PAY_INT                           0.040577
PAY_DOM                           0.032169
Name: 0, dtype: float64


SIV123                            0.140681
DM_P2                             0.117106
AB_CCARDS                         0.076253
AB_CWN                            0.052254
AB_AMF                            0.048927
ACCOPENING                        0.040577
PAY_DOM_BTB                       0.040577
PAY_DOM_URG                       0.040577
PAY_INT                           0.040577
PAY_DOM                           0.032169
Name: 0, dtype: float64